# Dynamic Programming via Policy Iteration 

In this notebook, a grid world task is solved using dynamic programming via Policy Iteration.

The code is adapted from NYU's 'Computational Cognitive Modeling' course to accomplish the homework assignment. 


# Outline
- [ 1 - Import Packages <img align="Right" src="https://raw.githubusercontent.com/jojozyang/CCM-site/master/homeworks/homework-RL/images/gridworld.png" width = 60%>](#1)
- [ 2 - Load the Environment](#2)
- [ 3 - Main functions](#3)
- [ 4 - Helper Functions](#4)
- [ 5 - Training](#5)

<a name="1"></a>
## 1 - Import Packages

In [15]:
import numpy as np
import random
import math
import statistics
from copy import deepcopy
from IPython.display import display, Markdown, Latex, HTML
from gridworld import GridWorld, random_policy

<a name="2"></a>
## 2 - Load the Environment

In [10]:
gridworld = [
       [ 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'x', 'g'],
       [ 'o', 'x', 'x', 'o', 'x', 'x', 'o', 'x', 'o'],
       [ 'o', 'x', 'x', 'o', 'x', 'x', 'o', 'x', 'o'],
       [ 'o', 'x', 'x', 'o', 'x', 'x', 'o', 'o', 'o'],
       [ 'o', 'x', 'x', 'o', 'x', 'x', 'x', 'o', 'o'],
       [ 's', 'o', 'o', 'o', 'o', 'o', 'o', 'o', 'x']
    ] # the problem described above, 'x' is a wall, 's' is start, 'g' is goal, and 'o' is a normal room

mygrid = GridWorld(gridworld)
mygrid.raw_print()  # print out the grid world
mygrid.index_print() # print out the indicies of each state
mygrid.coord_print() # print out the coordinates of each state (helpful in your code)

# define the rewards as a hash table (dictionary)
rewards={}

# mygrid.transitions contains all the pairwise state-state transitions allowed in the grid
# for each state transition intialize the reward to zero
for start_state in mygrid.transitions:
    for action in mygrid.transitions[start_state].keys():
        next_state = mygrid.transitions[start_state][action]
        rewards[str([start_state, action, next_state])] = 0.0

# now set the reward for moving up into state 8 (the goal state) to +10
rewards[str([17, 'up', 8])] = 10

# now set the penalty for walking off the edge of the grid and returning to state 45 (the start state)
for i in [0,1,2,3,4,5,6,7]:
    rewards[str([i, 'up', 45])] = -1
for i in [0,9,18,27,36,45]:
    rewards[str([i, 'left', 45])] = -1
for i in [45,46,47,48,49,50,51,52,53]:
    rewards[str([i, 'down', 45])] = -1
for i in [8,17,26,35,44,53]:
    rewards[str([i, 'right', 45])] = -1

## Welcome to your new Grid World!

**Raw World Layout**

o,o,o,o,o,o,o,x,g
o,x,x,o,x,x,o,x,o
o,x,x,o,x,x,o,x,o
o,x,x,o,x,x,o,o,o
o,x,x,o,x,x,x,o,o
s,o,o,o,o,o,o,o,x


**Indexes of each grid location as an id number**

0,1,2,3,4,5,6,7,8
9,10,11,12,13,14,15,16,17
18,19,20,21,22,23,24,25,26
27,28,29,30,31,32,33,34,35
36,37,38,39,40,41,42,43,44
45,46,47,48,49,50,51,52,53


**Indexes of each grid location as a tuple**

"(0,0)","(0,1)","(0,2)","(0,3)","(0,4)","(0,5)","(0,6)","(0,7)","(0,8)"
"(1,0)","(1,1)","(1,2)","(1,3)","(1,4)","(1,5)","(1,6)","(1,7)","(1,8)"
"(2,0)","(2,1)","(2,2)","(2,3)","(2,4)","(2,5)","(2,6)","(2,7)","(2,8)"
"(3,0)","(3,1)","(3,2)","(3,3)","(3,4)","(3,5)","(3,6)","(3,7)","(3,8)"
"(4,0)","(4,1)","(4,2)","(4,3)","(4,4)","(4,5)","(4,6)","(4,7)","(4,8)"
"(5,0)","(5,1)","(5,2)","(5,3)","(5,4)","(5,5)","(5,6)","(5,7)","(5,8)"


In [13]:
# initialize a new value table and a new random policy and uses functions provided in GridWorld to print these out in the notebook in a friendly way
value_table=np.zeros((mygrid.nrows,mygrid.ncols))
display(Markdown("**Current value table for each state**"))
mygrid.pretty_print_table(value_table)

policy_table = [[random_policy() for i in range(mygrid.ncols)] for j in range(mygrid.nrows)]
display(Markdown("**Current (randomized) policy**"))
mygrid.pretty_print_policy_table(policy_table)

**Current value table for each state**

0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0


**Current (randomized) policy**

↓,↑,→,↑,↓,←,→,▉,←
↑,▉,▉,→,▉,▉,→,▉,↑
↓,▉,▉,←,▉,▉,←,▉,→
↑,▉,▉,→,▉,▉,↑,↓,↑
←,▉,▉,↓,▉,▉,▉,←,→
↓,←,↓,→,←,↑,↓,↑,▉


<a name="3"></a>
## 3 - Main functions

In [11]:
def policy_evaluate(mygrid, value_table, policy_table, GAMMA):
    valid_states = list(mygrid.valid_states.keys())
    random.shuffle(valid_states)

    for state in valid_states:
        sx,sy = mygrid.index_to_coord(state)
        new_value = 0.0
        for action in mygrid.transitions[state].keys(): 
            p_sa = policy_table[sx][sy][action] 
            if p_sa == 1:
                next_state = mygrid.transitions[state][action]
                n_sx,n_sy = mygrid.index_to_coord(next_state)
                new_value = rewards[str([state,action,next_state])] + GAMMA * value_table[n_sx][n_sy]
        value_table[sx][sy] = new_value
        
# this is a helper function that will take a set of q-values and convert them into a greedy decision strategy        
def be_greedy(q_values):
    if len(q_values)==0:
        return {}
    
    keys = list(q_values.keys())
    vals = [q_values[i] for i in keys]    
    maxqs = [i for i,x in enumerate(vals) if x==max(vals)]
    if len(maxqs)>1:
        pos = random.choice(maxqs)
    else:
        pos = maxqs[0]
    policy = deepcopy(q_values)
    for i in policy.keys():
        policy[i]=0.0
    policy[keys[pos]]=1.0
    return policy
    
def policy_improve(mygrid, value_table, policy_table, GAMMA):
    # for each valid state
    valid_states = list(mygrid.valid_states.keys())
    for state in valid_states:
        # compute the Q-values for each action
        q_values = {}
        for action in mygrid.transitions[state].keys():
            next_state = mygrid.transitions[state][action]
            n_sx,n_sy = mygrid.index_to_coord(next_state)
            qval = rewards[str([state,action,next_state])] + GAMMA * value_table[n_sx][n_sy]
            q_values[action] = qval
        newpol = be_greedy(q_values) # take this dictionary and convert into a greedy policy
        # then update the policy table printing to allow more complex policies
        sx,sy = mygrid.index_to_coord(state)
        for action in mygrid.transitions[state].keys():
            policy_table[sx][sy][action] = newpol[action]

<a name="4"></a>
## 4 - Helper functions

<a name="5"></a>
## 5 - Train the agent

In [14]:
mygrid.pretty_print_table(value_table)
mygrid.pretty_print_policy_table(policy_table)

GAMMA = 0.9 # Discount factor 
for i in range(2000):
    policy_evaluate(mygrid, value_table, policy_table, GAMMA)
    policy_improve(mygrid, value_table, policy_table, GAMMA)

mygrid.pretty_print_table(value_table)
mygrid.pretty_print_policy_table(policy_table)

0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0


↓,↑,→,↑,↓,←,→,▉,←
↑,▉,▉,→,▉,▉,→,▉,↑
↓,▉,▉,←,▉,▉,←,▉,→
↑,▉,▉,→,▉,▉,↑,↓,↑
←,▉,▉,↓,▉,▉,▉,←,→
↓,←,↓,→,←,↑,↓,↑,▉


2.54187,2.8243,3.13811,3.48678,3.8742,4.30467,4.78297,0,0
2.28768,0,0,3.13811,0,0,5.31441,0,10
2.05891,0,0,2.8243,0,0,5.9049,0,9
2.28768,0,0,3.13811,0,0,6.561,7.29,8.1
2.54187,0,0,3.48678,0,0,0,6.561,7.29
2.8243,3.13811,3.48678,3.8742,4.30467,4.78297,5.31441,5.9049,0


→,→,→,→,→,→,↓,▉,←
↑,▉,▉,↑,▉,▉,↓,▉,↑
↓,▉,▉,↑,▉,▉,↓,▉,↑
↓,▉,▉,↓,▉,▉,→,→,↑
↓,▉,▉,↓,▉,▉,▉,↑,↑
→,→,→,→,→,→,→,↑,▉
